In [1]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
from sqlite3 import IntegrityError
from psycopg2 import errors

In [2]:
# read in the Excel file
tw_2016 = pd.read_csv('result20160101-20170101.csv', error_bad_lines=False)
tw_2017 = pd.read_csv('result20170101-20180101.csv', error_bad_lines=False)
tw_2018 = pd.read_csv('result20180101-20190101.csv', error_bad_lines=False)
tw_2019 = pd.read_csv('result20190101-20200101.csv', error_bad_lines=False)
tw_2020 = pd.read_csv('result20200101-20210101.csv', error_bad_lines=False)
tw_2021 = pd.read_csv('result20210101-20220101.csv', error_bad_lines=False)
tw_2022 = pd.read_csv('result20220101-20230101.csv', error_bad_lines=False)

b'Skipping line 1652: expected 34 fields, saw 67\n'


In [3]:
tw_list = [tw_2016, tw_2017, tw_2018, tw_2019, tw_2020, tw_2021, tw_2022]

In [4]:
#extract the cols we need and rename them
def extract_col(dataframe):
    tw_elem_name = dataframe[["user.id", "create_at", "text", "geo_tag.cityName", "geo_tag.stateName", "user.name"]]
    tw_elem_name = tw_elem_name.rename(columns={
            'user.id': 'user_id',
            'create_at': 'date',
            'text': 'tweet',
            'geo_tag.cityName': 'city',
            'geo_tag.stateName': 'state',
            'user.name': "user_name"
    })
    return tw_elem_name

In [5]:
def remove_dup(df):
    df = df.drop_duplicates(subset = ["user_id", "date", "tweet"], keep = "first")
    return df

In [6]:
tw_2016_select = remove_dup(extract_col(tw_2016))
tw_2017_select = remove_dup(extract_col(tw_2017))
tw_2018_select = remove_dup(extract_col(tw_2018))
tw_2019_select = remove_dup(extract_col(tw_2019))
tw_2020_select = remove_dup(extract_col(tw_2020))
tw_2021_select = remove_dup(extract_col(tw_2021))
tw_2022_select = remove_dup(extract_col(tw_2022))

In [7]:
dataframe_list = [tw_2016_select, tw_2017_select, tw_2018_select, tw_2019_select, tw_2020_select, tw_2021_select, tw_2022_select]
for item in dataframe_list:
    for index, r in item.iterrows():
        try: 
            # Connect to your PostgreSQL database
            conn = psycopg2.connect(
                    database="senior_project",
                    user="postgres",
                    password="1668",
                    host="localhost",
                    port="5433"
            )
            #create cursor obj
            cursor_obj = conn.cursor()
            cursor_obj.execute("INSERT INTO twitter (user_id, date, tweet, city, state, user_name) VALUES(%s, %s, %s, %s, %s, %s);", 
                               (r['user_id'], r['date'], r['tweet'], r['city'], r['state'], r['user_name']))
            # Commit the changes to the database
            conn.commit()
        except IntegrityError:
            pass
        except errors.UniqueViolation:
            pass
        finally:
            if conn:
                cursor_obj.close()
                # Close the database connection
                conn.close()

59861 records total